In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import *
from preprocessing import *
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
%load_ext autoreload

In [2]:

%autoreload 2

In [2]:
df_train_2024 = pd.read_csv('/home/toannn/PythonCode/ISIC2024/dataset/data2024/train-metadata.csv')
df_test_2024 = pd.read_csv('/home/toannn/PythonCode/ISIC2024/dataset/data2024/test-metadata.csv')
remove_columns = [col for col in df_train_2024.columns if (col not in df_test_2024.columns and col != "target")]
df_train_2024 = df_train_2024.drop(columns=remove_columns, axis=1)

# for isic_id in tqdm(df_train_2024["isic_id"]):
#     image_path = f"./dataset/data2024/image/{isic_id}.jpg"

#     if os.path.exists(image_path):
#         df_train_2024.loc[df_train_2024["isic_id"] == isic_id, "image_path"] = image_path
#     else:
#         print(f"Image {isic_id} not found")
#         break
num_cols = [
    'age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 
    'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 
    'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 
    'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
    'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM',
    'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
    'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
    'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
    'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z',
]

df_train_2024[num_cols] = df_train_2024[num_cols].fillna(df_train_2024[num_cols].median())
df_train, new_num_cols, new_cat_cols = feature_engineering(df_train_2024.copy())
df_test, _, _ = feature_engineering(df_test_2024.copy())

util_cols = ['isic_id', 'patient_id', 'target']
util_cols = util_cols + ["image_path"] if "image_path" in df_train_2024.columns else util_cols
num_cols += new_num_cols
# anatom_site_general
cat_cols = ["sex", "tbp_tile_type", "tbp_lv_location", "tbp_lv_location_simple"] + new_cat_cols
utils_col = ["target"]
train_cols =  cat_cols + num_cols + util_cols
df_train  = df_train[train_cols]

/tmp/ipykernel_684408/579303450.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_2024 = pd.read_csv('/home/toannn/PythonCode/ISIC2024/dataset/data2024/train-metadata.csv')


In [4]:
df_test_2024.target

AttributeError: 'DataFrame' object has no attribute 'target'

In [4]:
meta_handler = MetaDataClass(df_train)
category_encoder = OrdinalEncoder(
    categories='auto',
    dtype=int,
    handle_unknown='use_encoded_value',
    unknown_value=-2,
    encoded_missing_value=-1,
)
onhot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

cat_encoders = {'category_encoder': category_encoder, 'one_hot_encoder': onhot_encoder}

df_train = meta_handler.processCatAttr(df_train, cat_cols, 'train', cat_encoders, 'unknown')


/home/toannn/Documents/toanenv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
minmax_scale = MinMaxScaler()
num_encoders = {'minmax_scaler': minmax_scale}
df_train = meta_handler.processNumAttr(df_train, num_cols, 'train', num_encoders, 'mode')

Dropped color_uniformity column due to inf values


In [6]:
df_test = meta_handler.processCatAttr(df_test, cat_cols, 'test')
df_test = meta_handler.processNumAttr(df_test, num_cols, 'test')

In [7]:
#take colums without utils_col
meta_features = [col for col in df_train.columns if col not in utils_col]


In [8]:
df_train = meta_handler.split_data(df_train, 5,True,0.5)

Fold 0, size=12350, n_patient_id= 12350
  Train: index=[    0     1     2 ... 66148 66150 66151]
         group=['IP_4152176' 'IP_7299252' 'IP_1117889' ... 'IP_3710285' 'IP_9027701'
 'IP_9391134']
  Test:  index=[    5     6    14 ... 66141 66144 66149]
         group=['IP_4334766' 'IP_7312348' 'IP_6215230' ... 'IP_9025934' 'IP_1535132'
 'IP_4379891']
Fold 1, size=12816, n_patient_id= 12816
  Train: index=[    0     1     2 ... 66148 66149 66151]
         group=['IP_4152176' 'IP_7299252' 'IP_1117889' ... 'IP_3710285' 'IP_4379891'
 'IP_9391134']
  Test:  index=[    3     9    22 ... 66142 66143 66150]
         group=['IP_1196674' 'IP_3173681' 'IP_9027701' ... 'IP_1249536' 'IP_9577633'
 'IP_9027701']
Fold 2, size=11671, n_patient_id= 11671
  Train: index=[    1     2     3 ... 66149 66150 66151]
         group=['IP_7299252' 'IP_1117889' 'IP_1196674' ... 'IP_4379891' 'IP_9027701'
 'IP_9391134']
  Test:  index=[    0     4    11 ... 66132 66138 66146]
         group=['IP_4152176' 'IP_32604

In [9]:
df_train_x = df_train[meta_features]
df_train_y = df_train['target']
df_test_x = df_test[meta_features]
df_test_y = df_test['target']

KeyError: 'target'

In [ ]:
# write a simple neural network model to fit the df_train, and training it

class MetadataNet(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(MetadataNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size, seq_len, _ = x.shape
        out, hidden = self.lstm(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        out = out.view(batch_size, seq_len, -1)
        return out, hidden




In [ ]:
N_SPLITS = 5
gkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

if SUBSAMPLE:
    df_pos = df_train[df_train["target"] == 1]
    df_neg = df_train[df_train["target"] == 0]
    df_neg = df_neg.sample(frac=SUBSAMPLE_RATIO, random_state=42)
    df_train = pd.concat([df_pos, df_neg]).sample(frac=1.0, random_state=42).reset_index(drop=True)    

df_train["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train, df_train["target"], groups=df_train["patient_id"])):
    df_train.loc[val_idx, "fold"] = idx

def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

def custom_lgbm_metric(y_true, y_hat):
    # TODO: Refactor with the above.
    min_tpr = 0.80
    v_gt = abs(y_true-1)
    v_pred = np.array([1.0 - x for x in y_hat])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return "pauc80", partial_auc, True


def objective(trial):
    param = {
        "objective": "binary",
        # "metric": "custom",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "device": "gpu"
    }
    scores = []
    for fold in range(N_SPLITS):
        _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
        _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)
        dtrain = lgb.Dataset(_df_train[train_cols], label=_df_train["target"])
        gbm = lgb.train(param, dtrain)
        preds = gbm.predict(_df_valid[train_cols])
        score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
        scores.append(score)
    return np.mean(scores)

if OPTIMIZE_OPTUNA:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# lgb_params = {
#     'objective': 'binary',
#     # "random_state": 42,
#     "n_estimators": 1500,
#     'learning_rate': 0.001,
#     'bagging_freq': 1,
#     'pos_bagging_fraction': 0.75,
#     'neg_bagging_fraction': 0.05,
#     'feature_fraction': 0.6,
#     'lambda_l1': 0.2,
#     'lambda_l2': 0.7,
#     'num_leaves': 35,
#     "min_data_in_leaf": 50,
#     "verbosity": -1,
#     "device": "gpu"
#     # "extra_trees": True
# }
# new_params =  {
#     "objective": "binary",
#     "verbosity": -1,
#     "boosting_type": "gbdt",
#     "n_estimators": 2000,
#     'learning_rate': 0.03,    
#     'lambda_l1': 0.0004681884533249742, 
#     'lambda_l2': 8.765240856362274, 
#     'num_leaves': 136, 
#     'feature_fraction': 0.5392005444882538, 
#     'bagging_fraction': 0.9577412548866563, 
#     'bagging_freq': 6,
#     'min_child_samples': 60,
#     "device": "cpu"
# }
new_params = {
    "objective": "binary",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators": 200,
    'learning_rate': 0.05,    
    'lambda_l1': 0.0004681884533249742, 
    'lambda_l2': 8.765240856362274, 
    'num_leaves': 136, 
    'feature_fraction': 0.5392005444882538, 
    'bagging_fraction': 0.9577412548866563, 
    'bagging_freq': 6,
    'min_child_samples': 60,
    "device": "gpu"
}
lgb_scores = []
lgb_models = []
oof_df = pd.DataFrame()
for fold in range(N_SPLITS):
    _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
    _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)
    # model = lgb.LGBMClassifier(**new_params)
    model = VotingClassifier([(f"lgb_{i}", lgb.LGBMClassifier(random_state=i, **new_params)) for i in range(3)], voting="soft")
    model.fit(_df_train[train_cols], _df_train["target"])
    preds = model.predict_proba(_df_valid[train_cols])[:, 1]
    score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
    print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
    lgb_models.append(model)
    oof_single = _df_valid[["isic_id", "target"]].copy()
    oof_single["pred"] = preds
    oof_df = pd.concat([oof_df, oof_single])

lgbm_score = comp_score(oof_df["target"], oof_df["pred"], "")
print(f"LGBM Score: {lgbm_score:.5f}")

if DISPLAY_FEATURE_IMPORTANCE:
    # Make sure that this is a single model, not voting classifier. Will handle that later on.
    importances = np.mean([model.feature_importances_ for model in lgb_models], 0)
    df_imp = pd.DataFrame({"feature": model.feature_name_, "importance": importances}).sort_values("importance").reset_index(drop=True)

    plt.figure(figsize=(16, 12))
    plt.barh(df_imp["feature"], df_imp["importance"])
    plt.show()

NameError: name 'SUBSAMPLE' is not defined